### This notebook is for focusing on a roll call to see how it is transcribed

Recognizing short words by different speakers is difficult.  This notebook focuses in a roll call vote to see if changing model parameters can improve it.  


In [ ]:
import sys
import pandas as pd
sys.path.append("../")
from pathlib import Path

### use ffmpeg to get a section of a meeting
This 30 second clip is a roll call vote

In [ ]:
import subprocess
from pathlib import Path

# Input and output file paths
input_file = Path("../data/video/regular_council_meeting___2025_02_26.mp4")
clip_file = Path("../data/video/regular_council_meeting___2025_02_26_clip_4-50_to_5-20.mp4")

# Parameters for clip extraction
start_time = "4:50"
duration = "30"  # 30 seconds

# Run FFmpeg command
result = subprocess.run(
    [
        "ffmpeg",
        "-i",
        str(input_file),
        "-ss",
        start_time,
        "-t",
        duration,
        "-c",
        "copy",  # Copy codec (fast but might not be frame accurate)
        "-avoid_negative_ts",
        "1",
        str(clip_file),
        "-y",  # Overwrite if exists
    ],
    capture_output=True,
    text=True,
)

# Check if command was successful
if result.returncode == 0:
    print(f"Clip successfully extracted to: {clip_file}")
else:
    print(f"Error extracting clip: {result.stderr}")

### experiment with model parameters

using these setting actually made the results worse:
- min_speakers=3,  # Specify at least 3 speakers
- max_speakers=15,  # Limit to at most 10 speakers
- diarize_min_duration=0.1,  # Shorter minimum segment duration
I also tested with medium, and large versions but the results using tiny were the same


In [ ]:
from src.videos import transcribe_video_with_diarization

transcription_dir = Path("../data/transcripts")

transcript_data = await transcribe_video_with_diarization(
    clip_file,
    transcription_dir,
    model_size="tiny",
)

In [ ]:
def format_timestamp(seconds: float) -> str:
    """Convert seconds to HH:MM:SS format"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"


from ipywidgets import HTML, VBox, Layout
from textwrap import fill

# Create formatted HTML output
html_output = ["<h3>Meeting Script</h3>"]
html_output.append("<hr>")

current_speaker = None
current_text = []
current_start = None

for segment in transcript_data["segments"]:
    if current_speaker != segment["speaker"]:
        # Output previous speaker's text
        if current_speaker:
            timestamp = format_timestamp(current_start)
            wrapped_text = fill(" ".join(current_text), width=80)
            html_output.append(f"<p><b>[{timestamp}] {current_speaker}:</b><br>")
            html_output.append(f"{wrapped_text}</p>")
            html_output.append("<hr>")

        # Start new speaker
        current_speaker = segment["speaker"]
        current_text = [segment["text"].strip()]
        current_start = segment["start"]
    else:
        # Continue current speaker
        current_text.append(segment["text"].strip())

# Output final speaker
if current_speaker:
    timestamp = format_timestamp(current_start)
    wrapped_text = fill(" ".join(current_text), width=80)
    html_output.append(f"<p><b>[{timestamp}] {current_speaker}:</b><br>")
    html_output.append(f"{wrapped_text}</p>")
    html_output.append("<hr>")

# Display formatted output
display(
    HTML(
        value="".join(html_output),
        layout=Layout(width="100%", border="1px solid gray", padding="10px"),
    )
)